We are analyzing a 3-story building subjected to a random earthquake acceleration. We need to find the peak floor displacements, inter-story drifts, and identify if any floor exceeds the "Safe Drift Limit."

In [14]:
import numpy as np

#THE STRUCTURAL MODEL (System Properties) 
# Floor masses (kg) and Stiffness (N/m) for a 3-story structure
masses = np.array([25000, 25000, 20000], dtype='float64') 
stiffness = np.array([3e7, 2.5e7, 2e7]) 
print("Floor Masses (kg):", masses)
print("Floor Stiffness (N/m):", stiffness)


Floor Masses (kg): [25000. 25000. 20000.]
Floor Stiffness (N/m): [30000000. 25000000. 20000000.]


In [13]:
# Initialize Global Stiffness Matrix [K] using NumPy Indexing
K = np.zeros((3, 3))
K[0,0], K[0,1] = stiffness[0] + stiffness[1], -stiffness[1]
K[1,0], K[1,1], K[1,2] = -stiffness[1], stiffness[1] + stiffness[2], -stiffness[2]
K[2,1], K[2,2] = -stiffness[2], stiffness[2]
print("Global Stiffness Matrix [K]:\n", K)

Global Stiffness Matrix [K]:
 [[ 55000000. -25000000.         0.]
 [-25000000.  45000000. -20000000.]
 [        0. -20000000.  20000000.]]


In [16]:
# GROUND MOTION SIMULATION (Random Module & Slicing) 
np.random.seed(42)
dt = 0.01 # Time step
time = np.arange(0, 10, dt) 
# Generate raw earthquake noise and scale by 0.3g (Seismic Coefficient)
raw_accel = np.random.normal(0, 0.5, size=time.shape) 
g = 9.81
ground_accel = raw_accel * (0.3 * g) 
print("Simulated Ground Acceleration (first 10 values):", ground_accel[:10])
print("Time Array (first 10 values):", time[:10])
print("Time Step (dt):", dt)



Simulated Ground Acceleration (first 10 values): [ 0.73091488 -0.20345592  0.95307368  2.24113843 -0.34455669 -0.34453253
  2.32381166  1.1292802  -0.69083156  0.7983771 ]
Time Array (first 10 values): [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]
Time Step (dt): 0.01


In [ ]:
# FORCE CALCULATION (Broadcasting - NO LOOPS ALLOWED) 
# Force = Mass * Acceleration. 
# We use masses[:, np.newaxis] to broadcast 1D mass over 1D acceleration to get 2D force matrix
floor_forces = masses[:, np.newaxis] * ground_accel # Shape: (3 floors, 1000 steps)
print("Floor Forces Matrix Shape:", floor_forces.shape)


Floor Forces Matrix Shape: (3, 1000)


In [ ]:
 #SOLVING DISPLACEMENTS (Matrix Operations) 
# U = K^-1 * F (Static Equivalent for demonstration of Linear Algebra)
K_inv = np.linalg.inv(K)
displacements = np.dot(K_inv, floor_forces) 
# Instantaneous solve for all time steps
print("Displacements Matrix Shape:", displacements.shape)

Displacements Matrix Shape: (3, 1000)


In [21]:
# STATISTICAL ANALYSIS & SEARCHING -
# Find Peak Ground Acceleration (PGA)
pga = np.max(np.abs(ground_accel))
peak_idx = np.argmax(np.abs(ground_accel))
print("Peak Ground Acceleration (PGA): {:.4f} m/s² at time {:.2f} s".format(pga, time[peak_idx]))
print("Displacements at PGA time (m):", displacements[:, peak_idx])

Peak Ground Acceleration (PGA): 5.6693 m/s² at time 2.09 s
Displacements at PGA time (m): [0.01322835 0.02343308 0.02910238]


In [22]:
# Calculate Inter-story Drift (Difference between adjacent floor displacements)
# Using NumPy Difference (axis=0 for floors)
drifts = np.diff(displacements, axis=0, prepend=0) 

# Logic Search: Find moments where drift exceeds 0.005m (Critical Safety Limit)
danger_zones = np.where(np.abs(drifts) > 0.005)

#  AGGREGATION & LOGGING (Statistical Operations) 
mean_response = np.mean(displacements, axis=1)
max_response = np.max(np.abs(displacements), axis=1)

In [23]:

print(f"Peak Ground Acceleration (PGA): {pga:.3f} m/s^2 at {time[peak_idx]:.2f}s")
print(f"Max Roof Displacement: {max_response[2]*1000:.2f} mm")
print(f"Safety Check: Found {len(danger_zones[0])} instances of excessive drift.")
if max_response[2] > 0.05:
    print("STATUS: STRUCTURE TRASH (Exceeds 50mm limit). RE-DESIGN STIFFNESS.")
else:
    print("STATUS: STRUCTURE STABLE.")

Peak Ground Acceleration (PGA): 5.669 m/s^2 at 2.09s
Max Roof Displacement: 29.10 mm
Safety Check: Found 192 instances of excessive drift.
STATUS: STRUCTURE STABLE.
